### Homework 9

이번 과제에서는 Kalman Filtering, Smoothering을 비롯해 평활화에 관련된 문제를 실습해 보도록 한다

#### 1. Data Import

`pandas.read_csv()`를 이용해 아래의 두 가지 데이터를 준비하도록 한다

- Gold_USD.csv : 일별 금 가격 (Commodity)
- USD_KRW.csv : 일별 원-달러 환율

In [ ]:
import numpy as np
import pandas as pd

# write your code here!

**1.1 Data Preprocessing**

준비된 데이터는 최근 날짜에서 과거날짜로, 즉, 역순으로 정렬되어 있다. 적절한 방법을 이용하여 데이터를 과거-현재 순으로 정렬하라

In [ ]:
# write your code here!

**1.2 Datetime Index**

분석의 편리함을 위해 준비된 데이터의 index를 적절한 방법을 이용해 datetime index로 변환하라. pandas에는 이를 위한 method가 정의되어 있다 (검색을 추천함)

In [1]:
# write your code here!

#### 2. State Space models

**2.1 $\mathrm{ARMA}$ models**

두 자료에 대하여 적절한 차수를 탐색해 $\mathrm{ARMA}$모형을 추정하라. 단, 가격은 종가를 기준으로 한다

In [ ]:
# write your code here!

**2.2 Kalman Filtering**

준비된 `SpectralAnalysisStateSpaceModel.py`를 활용하여 Kalman Filtering을 수행하라. **2.1**에서 추정한 $\mathrm{ARMA}$모형과 비교하라. $\mathrm{ARMA}$모형은 state space model이라고 볼 수 있는가?

In [ ]:
# write your code here!

#### 3. Kalman Smoothing

**3.1 Estimate Kalman Smoother**

아래의 함수를 이용하여 USD/KRW과 GOLD 가격에 대해 Kalman Smoothing을 진행하라. 단, q = 0.001로 지정한다.

In [ ]:
from statsmodels.tsa.statespace.mlemodel import MLEModel
def kalman_smoothing(close : pd.Series, 
                     q : float, dt : float = 1.0, s : float = 0.5,
                     A : np.array = np.array([[1]]), 
                     Q : np.array = np.array([[1]]), 
                     H : np.array = np.array([[1]]), 
                     R : np.array = np.eye(1),
                     m0 : np.array = np.array([[1]]),
                     P0 : np.array = np.eye(1),
                     k_state : int = 1) -> pd.DataFrame :
    Q = q * Q
    R = s ** 2 * R

    kf = MLEModel(close, k_states = k_state)
    kf._state_names = ['x1']
    kf['design'] = H
    kf['obs_cov'] = R
    kf['transition'] = A
    kf['selection'] = np.eye(1)
    kf['state_cov'] = Q

    kf.initialize_known(A @ m0[:, 0], A @ P0 @ A.T + Q)
    res = kf.smooth(params = [])
    output = pd.DataFrame(res.states.smoothed.x1.values, index = close.index, columns = ['smoothered'])
    
    return output

In [ ]:
# write your code here!

**3.2 plotting**

kalman smoothe된 두 변수를 plotting하라. 비슷한 추세를 가지고 있는가?

In [ ]:
# write your code here!

#### 4. Kernel Smoothing

Locally Weighted Scatterplot Smoothing(LOESS, 국소회귀평활화)은 smoothing방법 중 하나이다. 자세한 내용은 아래 article를 참조하도록 하자

https://towardsdatascience.com/loess-373d43b03564

`statsmodels.api.nonparametric.lowess`에서는 LOESS평활화를 추정할 수 있다. `lowess()`함수의 대표적인 parameter는 아래와 같다

```python
statsmodels.nonparametric.smoothers_lowess.lowess(
    endog, 
    exog, 
    frac = 0.6666666666666666, 
    it=3, 
    delta=0.0, 
    xvals=None, 
    is_sorted=False, 
    missing='drop', 
    return_sorted=True)
```

endog에는 가격계열의 pandas.Series를 지정하고, exog에는 `np.arange(len(pd.Series))`를 지정하면 된다. `frac`은 평활화에 가장 중요한 역할을 하는 parameter로, **지정한 값 만큼 local band를 지정하여 회귀를 진행**한다. 예를 들어, `frac=1`인 경우 전 구간에서 회귀를 진행하는 것이고, `frac=0.5`를 지정하면 전 구간에서 절반의 기간만큼 회귀를 진행한다

`frac=0.1`을 지정하여 USD/KRW과 GOLD가격 데이터에 대한 smoothing을 진행하고 3번의 결과와 비교하라. 어떠한 차이가 있는가?

In [ ]:
import statsmodels.api as sm

# write your code here!

#### 5. Structural Change

Smoothing의 가장 큰 장점은 시장의 국면을 쉽게 판단할 수 있다는 것이다. smoothing 결과로 본 USD/KRW 시장과 GOLD가격 시장은 어느 시기가 활황이었고, 어느 시기가 불황이었는가?
이 시기는 전세계적으로 경기 침체가 일어난 시기와 어떠한 관계가 있는가?